In [1]:
import tensorflow as tf

In [9]:
import numpy as np

In [2]:
( x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape( -1, 28, 28, 1  )
x_test = x_test.reshape( -1, 28, 28, 1 )

y_train = tf.keras.utils.to_categorical( y_train, 10 )
y_test = tf.keras.utils.to_categorical( y_test, 10 )
print( x_train.shape, y_train.shape )

(60000, 28, 28, 1) (60000, 10)


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
def StackLayers( model ):
    # layer 1
    model.add( tf.keras.layers.Conv2D( filters = 32, kernel_size = [ 3, 3 ],
                                   padding = 'SAME', activation = 'relu' ) )
    model.add( tf.keras.layers.MaxPooling2D( pool_size = [ 2, 2 ],
                                        padding = 'SAME', strides = 2 ) )
    model.add( tf.keras.layers.Dropout( 0.3 ) )

    # layer 2
    model.add( tf.keras.layers.Conv2D( filters = 64, kernel_size = [ 3, 3 ],
                                  padding = 'SAME', activation = 'relu' ) )
    model.add( tf.keras.layers.MaxPooling2D( pool_size = [ 2, 2 ],
                                        padding = 'SAME', strides = 2 ) )
    model.add( tf.keras.layers.Dropout( 0.3 ) )

    # layer 3
    model.add( tf.keras.layers.Conv2D( filters = 128, kernel_size = [ 3, 3 ],
                                  padding = 'SAME', activation = 'relu' ) )
    model.add( tf.keras.layers.MaxPooling2D( pool_size = [ 2, 2 ],
                                        padding = 'SAME', strides = 2 ) )
    model.add( tf.keras.layers.Dropout( 0.3 ) )

    model.add( tf.keras.layers.Flatten() )
    model.add( tf.keras.layers.Dense( units = 625, activation = 'relu' ) )
    model.add( tf.keras.layers.Dropout( 0.5 ) )

    model.add( tf.keras.layers.Dense( units = 10, activation = 'softmax' ) )
    
def CompileModel( model ):
    model.compile( loss = 'categorical_crossentropy',
           optimizer = tf.keras.optimizers.Adam( learning_rate ), metrics = [ 'accuracy' ] )
    
def TrainingModel( model ):
    model.fit( x_train, y_train, batch_size, training_epochs )
    

In [5]:
models = []
num_models = 5
for m in range( num_models ):
    models.append( tf.keras.Sequential() )
    StackLayers( models[m] )
    CompileModel( models[m] )

In [6]:
for m in models:
    TrainingModel( m )

Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 73us/sample - loss: 0.2646 - accuracy: 0.9135
Epoch 2/15
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0845 - accuracy: 0.9729
Epoch 3/15
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0652 - accuracy: 0.9795
Epoch 4/15
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0543 - accuracy: 0.9827
Epoch 5/15
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0488 - accuracy: 0.9850
Epoch 6/15
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0423 - accuracy: 0.9871
Epoch 7/15
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0395 - accuracy: 0.9877
Epoch 8/15
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0383 - accuracy: 0.9878
Epoch 9/15
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0369 - accuracy: 0.9883
Epoch 10/15
60

In [11]:
test_size = len( y_test )
predictions = np.zeros( [ test_size, 10 ] )
for m_idx, m in enumerate( models ):
    print( m_idx, 'Accuracy: ', m.evaluate( x_test, y_test )[1] )
    p = m.predict( x_test )
    predictions += p

10000/10000 [==============================] - 1s 64us/sample - loss: 0.0236 - accuracy: 0.9933
0 Accuracy:  0.9933
10000/10000 [==============================] - 1s 70us/sample - loss: 0.0206 - accuracy: 0.9944
1 Accuracy:  0.9944
10000/10000 [==============================] - 1s 63us/sample - loss: 0.0260 - accuracy: 0.9913
2 Accuracy:  0.9913
10000/10000 [==============================] - 1s 63us/sample - loss: 0.0209 - accuracy: 0.9937
3 Accuracy:  0.9937
10000/10000 [==============================] - 1s 62us/sample - loss: 0.0185 - accuracy: 0.9943
4 Accuracy:  0.9943


In [14]:
ensemble_correct_prediction = tf.equal(
                                      tf.argmax( predictions, 1 ), tf.argmax( y_test, 1 ) )
ensemble_accuracy = tf.reduce_mean(
                                  tf.cast( ensemble_correct_prediction, tf.float32 ) )
tf.print( 'Ensembel accuracy: ', ensemble_accuracy * 100, '%' )

Ensembel accuracy:  99.56 %
